In computer vision, Intersection over Union (IoU) is a metric used to evaluate the accuracy of an object detector on a particular dataset. It is a measure of the overlap between two bounding boxes: the predicted bounding box and the ground truth bounding box.

The IoU is calculated as follows:

  > IoU = Intersection/Union

- **Area of Intersect**: The area where the predicted bounding box and the ground truth bounding box intersect.
- **Area of Union**: The total area covered by both the predicted bounding box and the ground truth bounding box.

IoU values range from 0 to 1, where:
- An IoU of 0 means there is no overlap between the predicted and ground truth bounding boxes.
- An IoU of 1 means the predicted and ground truth bounding boxes perfectly overlap.

IoU is commonly used in tasks like object detection to determine how well the predicted bounding boxes match the actual objects in the image. A higher IoU indicates a better prediction.


If we have points 

Ground-Truth-Box => box1 =  [x1,y1,x2,y2]
Detected-Box => box2 = [x1,y1,x2,y2]

- Assume top-left of image is origin

(box1.x1,box1.y1) => top-left of predicted box
(box1.x2,box1.y2) => bottom-right of pridicted box

(box2.x1,box2.y1) => top-left of ground-truth box
(box2.x2,box2.y2) => bottom-right of ground-truth box

ix1 = max(box1.x1,box2.x1)
iy1 = max(box1.y1,box2.y1)

ix2 = min(box1.x2,box2.x2)
iy2 = min(box1.y2,box2.y2)

Intersection-Box => box3 = [ix1,iy1,ix2,iy2]

Area-of-Intersection = abs(iy2-iy1) * abs(ix2-ix1)
Area-of-Union = Area(GTB) + Area(DTB) - Area-of-Intersection

## Implementation

In [ ]:
import torch

def intersection_over_union(predictions,labels,format):
  '''
  Calculates intersection over union 
  
  Parameters:
    predictions: predictions of bounding boxes
    lables: correct labels of boxes
    format: midpoint/corners (x,y,w,h) or (x1,y1,x2,y2)
  '''

  if format == "midpoint":
      box1_x1 = predictions[..., 0:1] - predictions[..., 2:3] / 2
      box1_y1 = predictions[..., 1:2] - predictions[..., 3:4] / 2
      box1_x2 = predictions[..., 0:1] + predictions[..., 2:3] / 2
      box1_y2 = predictions[..., 1:2] + predictions[..., 3:4] / 2
      box2_x1 = labels[..., 0:1] - labels[..., 2:3] / 2
      box2_y1 = labels[..., 1:2] - labels[..., 3:4] / 2
      box2_x2 = labels[..., 0:1] + labels[..., 2:3] / 2
      box2_y2 = labels[..., 1:2] + labels[..., 3:4] / 2

  elif format == "corners":
      box1_x1 = predictions[..., 0:1]
      box1_y1 = predictions[..., 1:2]
      box1_x2 = predictions[..., 2:3]
      box1_y2 = predictions[..., 3:4]
      box2_x1 = labels[..., 0:1]
      box2_y1 = labels[..., 1:2]
      box2_x2 = labels[..., 2:3]
      box2_y2 = labels[..., 3:4]

  x1 = torch.max(box1_x1, box2_x1)
  y1 = torch.max(box1_y1, box2_y1)
  x2 = torch.min(box1_x2, box2_x2)
  y2 = torch.min(box1_y2, box2_y2)

  # clamp(0) for boxes with no intersection  
  intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)
  box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
  box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

  return intersection / (box1_area + box2_area - intersection + 1e-6)